In [6]:
#
# classifer notebook
#
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Lambda, Compose
from LandmarkDataset import LandmarkDataset

#
# MODEL
#

class NN(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)
        #self.fc2 = torch.nn.Linear(hidden_size, 48)
        #self.fc3 = torch.nn.Linear(48, num_classes)
        self.dropout = torch.nn.Dropout(p=0.05)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        #out = self.relu(out)
        #out = self.dropout(out)
        
        #out = self.fc3(out)
        
        return out

    
#
# PARAMS
#

batch_size = 4
hidden_size = 96
learning_rate = .01
num_epochs = 250


#
# INITIAL DATA
#

# for directory load each file
# generate mapping of file -> class -> idx

transformations = Compose([
    Lambda(lambda x: torch.tensor(x.values).float())
])

dataset = LandmarkDataset("/home/jovyan/train/data/",
                          transform=transformations)

num_classes = dataset.num_class
input_size = dataset.input_size #2 * (21 * 3) + 12 #138

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)



#
# INIT MODELS, LOSS FN, GRAD
#

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NN(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

#
# TRAIN
#

running_loss_epoch = 0
for epoch in range(num_epochs):
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  

        # zero out accumulated gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(landmarks)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        # NB: len(dataloader) is num of batches
        running_loss_epoch += loss.item()
        if ((epoch+1) % 10 == 0) and ((batch_idx+1) % len(dataloader) == 0):
            print ('Epoch [{}/{}], Loss: {:.4f}, Running Loss (per N epoch): {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item(), running_loss_epoch))
            running_loss_epoch = 0


Epoch [10/250], Loss: 1.2211, Running Loss (per N epoch): 148.8787
Epoch [20/250], Loss: 0.3399, Running Loss (per N epoch): 81.5112
Epoch [30/250], Loss: 0.4593, Running Loss (per N epoch): 41.8602
Epoch [40/250], Loss: 0.0162, Running Loss (per N epoch): 32.1291
Epoch [50/250], Loss: 0.0179, Running Loss (per N epoch): 23.0621
Epoch [60/250], Loss: 0.2397, Running Loss (per N epoch): 21.6884
Epoch [70/250], Loss: 0.0198, Running Loss (per N epoch): 17.9256
Epoch [80/250], Loss: 0.1140, Running Loss (per N epoch): 18.0688
Epoch [90/250], Loss: 0.0170, Running Loss (per N epoch): 8.9599
Epoch [100/250], Loss: 0.0000, Running Loss (per N epoch): 12.2236
Epoch [110/250], Loss: 0.0702, Running Loss (per N epoch): 9.1524
Epoch [120/250], Loss: 0.0000, Running Loss (per N epoch): 8.9565
Epoch [130/250], Loss: 0.0151, Running Loss (per N epoch): 7.7721
Epoch [140/250], Loss: 0.0210, Running Loss (per N epoch): 6.3685
Epoch [150/250], Loss: 3.4608, Running Loss (per N epoch): 13.7385
Epoch [1

In [8]:
#
# EXPORT
#

dummy_input = torch.zeros(input_size)
#model.load_state_dict(torch.load('./model_overfit.pt'))
torch.onnx.export(model, dummy_input, 'onnx_model.onnx', export_params=True,
                  input_names = ['landmarks'], output_names = ['class'], verbose=True)

# Export idx-class map per run
dataset_labels = pd.DataFrame( dataset.class_map.items(), columns=["idx", "label"] )
dataset_labels.to_csv("label_map.csv", index=False)

#import onnx
#onnx_model = onnx.load("./onnx_model.onnx")
#onnx.checker.check_model(onnx_model)


Exported graph: graph(%landmarks : Float(138, strides=[1], requires_grad=0, device=cpu),
      %fc1.bias : Float(96, strides=[1], requires_grad=1, device=cpu),
      %fc2.bias : Float(6, strides=[1], requires_grad=1, device=cpu),
      %onnx::MatMul_12 : Float(138, 96, strides=[1, 138], requires_grad=0, device=cpu),
      %onnx::MatMul_13 : Float(96, 6, strides=[1, 96], requires_grad=0, device=cpu)):
  %onnx::Add_6 : Float(96, strides=[1], device=cpu) = onnx::MatMul[onnx_name="MatMul_0"](%landmarks, %onnx::MatMul_12) # /opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  %input : Float(96, strides=[1], requires_grad=1, device=cpu) = onnx::Add[onnx_name="Add_1"](%fc1.bias, %onnx::Add_6) # /opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  %input.3 : Float(96, strides=[1], requires_grad=1, device=cpu) = onnx::Relu[onnx_name="Relu_2"](%input) # /opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1453:0
  %onnx::Add_10 : Float(6, str

In [3]:
#
# EVAL
#
torch.set_printoptions(precision=4, sci_mode=False)
softmax = torch.nn.Softmax(dim=1)
accuracy = 0
count = 0
with torch.no_grad():
    
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  
        out = model(landmarks)
        prob = softmax(out.data)     #setup for threshold or 'garbage' class
        _, klass = torch.max(out.data, 1)
        
        print(klass, labels, klass==labels)
        print(prob)
        print("-----")
        
        accuracy += (klass == labels).sum().item()
        count += len(labels)
        

print('--------')
print("Accuracy {}/{} : {:.4f}".format(accuracy, count, accuracy/count))


tensor([2, 2, 4, 5]) tensor([2, 2, 4, 5]) tensor([True, True, True, True])
tensor([[    0.0000,     0.0009,     0.9991,     0.0000,     0.0000,     0.0000],
        [    0.0000,     0.0001,     0.9999,     0.0000,     0.0000,     0.0000],
        [    0.0000,     0.0000,     0.0000,     0.0001,     0.9998,     0.0000],
        [    0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     1.0000]])
-----
tensor([0, 1, 0, 1]) tensor([0, 1, 0, 1]) tensor([True, True, True, True])
tensor([[    0.9999,     0.0001,     0.0000,     0.0000,     0.0000,     0.0000],
        [    0.0000,     0.9978,     0.0022,     0.0000,     0.0000,     0.0000],
        [    1.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000],
        [    0.0000,     1.0000,     0.0000,     0.0000,     0.0000,     0.0000]])
-----
tensor([5, 0, 5, 0]) tensor([5, 0, 5, 0]) tensor([True, True, True, True])
tensor([[    0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     1.0000],
        [    1.0000,   

In [4]:
print("Done")

Done
